In [23]:
import os
os.chdir(r'D:\8th Semester\Machine learning\ML Labs')


In [24]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Dense, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [25]:
class PC:
    @staticmethod
    def build(time_steps, num_features, reg=0.0005):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        # define the model input and first CONV module
        inputs = Input(shape=(time_steps, num_features))
        lstm=Bidirectional(LSTM(48,return_sequences=True))(inputs)
        lstm=LSTM(48)(lstm)
        lstm=Activation('tanh')(lstm)
        lstm=Dense(24)(lstm)
        # create the model
        model = Model(inputs, lstm, name="PC")
        
        # return the constructed network architecture
        return model

In [26]:
model1 =  PC.build(time_steps=24, num_features=21, reg=0.005)

In [27]:
model1.summary()

Model: "PC"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 24, 96)              │          26,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 48)                  │          27,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 24)                  │           1,176 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 55,896 (218.34 KB)

 Trainable params: 55,896 (218.34 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab2\\E3-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'D:\8th Semester\Machine learning\ML Labs\lab2'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\h2istory.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\h2istory.json"])

In [29]:
model = None
start_epoch = 0
time_steps=24
num_features=21

In [30]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [31]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [33]:
import os
path_dataset =r'D:\8th Semester\Machine learning\ML Labs\DATASET'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [ ]:
time_steps=24
num_features=21

In [ ]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=24)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=24)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=24)
print('Time Consumed', time.time()-start, "sec")

In [ ]:
epochs = 2
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

In [38]:
import numpy as np
from keras.models import load_model
from keras.metrics import MeanAbsoluteError
from keras.saving import register_keras_serializable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Simulated 3D input data: (samples, time_steps=24, features=21)
X = np.random.rand(1000, 24, 21)   # replace with your actual data
y = np.random.rand(1000, 1)

# Scaling features
scaler = MinMaxScaler()
X_reshaped = X.reshape(-1, X.shape[2])       # reshape for scaling
X_scaled = scaler.fit_transform(X_reshaped)
X_scaled = X_scaled.reshape(-1, 24, 21)       # reshape back to original

y_scaled = scaler.fit_transform(y)

# Split into train/test
train_X, test_X, train_y, test_y = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Fix for loading 'mae' metric
@register_keras_serializable(package='Custom', name='mae')
def mae(y_true, y_pred):
    return MeanAbsoluteError()(y_true, y_pred)

# Load model with fix
model = load_model(
    r'D:\8th Semester\Machine learning\ML Labs\lab2\E3-cp-0002-loss0.04.h5',
    custom_objects={'mae': mae}
)

# Reshape test_X to match model input (if needed)
test_X = test_X.reshape(-1, 24, 21)

# Predict and inverse transform
y_pred_scaled = model.predict(test_X)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Metrics
MAE = np.mean(np.abs(y_pred - y_test_unscaled))
MEDAE = np.median(np.abs(y_pred - y_test_unscaled))
MSE = np.mean(np.square(y_pred - y_test_unscaled))
RMSE = np.sqrt(MSE)
MAPE = np.mean(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
MDAPE = np.median(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100

# Print
print('Mean Absolute Error (MAE):', round(MAE, 2))
print('Median Absolute Error (MedAE):', round(MEDAE, 2))
print('Mean Squared Error (MSE):', round(MSE, 2))
print('Root Mean Squared Error (RMSE):', round(RMSE, 2))
print('Mean Absolute Percentage Error (MAPE):', round(MAPE, 2), '%')
print('Median Absolute Percentage Error (MDAPE):', round(MDAPE, 2), '%')
print('\ny_test_unscaled.shape =', y_test_unscaled.shape)
print('y_pred.shape =', y_pred.shape)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step 
Mean Absolute Error (MAE): 0.26
Median Absolute Error (MedAE): 0.24
Mean Squared Error (MSE): 0.09
Root Mean Squared Error (RMSE): 0.31
Mean Absolute Percentage Error (MAPE): 680.57 %
Median Absolute Percentage Error (MDAPE): 40.79 %

y_test_unscaled.shape = (200, 1)
y_pred.shape = (200, 24)


In [39]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab2\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'D:\8th Semester\Machine learning\ML Labs\lab2\E3-cp-0002-loss0.04.h5'
start_epoch= 17

In [44]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

# Define mae explicitly
def mae(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))

# Dummy placeholders for data (replace with actual data)
# test_X and test_y should be defined properly
# scaler should be fitted on training data and reused here
test_X = np.random.rand(100, 24, 21)
test_y = np.random.rand(100, 1)
scaler = MinMaxScaler()
scaler.fit(test_y)

# Model checkpoint path
checkpoints = 'best_model.h5'

# Check if model path exists
model_path = r'D:\8th Semester\Machine learning\ML Labs\lab2\E3-cp-0002-loss0.04.h5'
if os.path.exists(model_path):
    print("[INFO] Loading model from:", model_path)
    model = load_model(model_path, custom_objects={'mae': mae})

    # Update learning rate
    print("[INFO] old learning rate:", model.optimizer.learning_rate.numpy())
    model.optimizer.learning_rate.assign(1e-4)
    print("[INFO] new learning rate:", model.optimizer.learning_rate.numpy())
else:
    print("[INFO] No model found. Define and compile your model here.")
    # Example placeholder model structure
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    model = Sequential([
        LSTM(64, input_shape=(24, 21)),
        Dense(1)
    ])
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=['mae', 'mape'])

# Define callbacks
EpochCheckpoint1 = ModelCheckpoint(checkpoints, monitor="val_loss", save_best_only=True, verbose=1)
callbacks = [EpochCheckpoint1]

# Train the model (dummy example; replace with real data)
# history = model.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=20, callbacks=callbacks)

# Make predictions
y_pred_scaled = model.predict(test_X)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Print shape to confirm
print("Predicted shape:", y_pred.shape)
print("Actual shape:", y_test_unscaled.shape)


[INFO] Loading model from: D:\8th Semester\Machine learning\ML Labs\lab2\E3-cp-0002-loss0.04.h5
[INFO] old learning rate: 0.001
[INFO] new learning rate: 1e-04
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step
Predicted shape: (100, 24)
Actual shape: (100, 1)


[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [47]:
# ===============================
# 1. Imports and Setup
# ===============================
import numpy as np
import tensorflow as tf
from keras.models import load_model
from keras.metrics import MeanAbsoluteError
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.saving import register_keras_serializable

# Enable eager execution
tf.config.run_functions_eagerly(True)

# Fix for the 'mae' metric during model loading
@register_keras_serializable(package='Custom', name='mae')
def mae(y_true, y_pred):
    return MeanAbsoluteError()(y_true, y_pred)

# ===============================
# 2. Dummy Data Setup (Replace with your actual data)
# ===============================
# Assume 24 time steps and 21 features as required by model
train_X = np.random.rand(1000, 24, 21)
train_y = np.random.rand(1000, 1)
validation_X = np.random.rand(200, 24, 21)
validation_y = np.random.rand(200, 1)

# Scaler (for inverse transform later)
scaler = MinMaxScaler()
scaler.fit(train_y)

# ===============================
# 3. Load Model
# ===============================
model_path = r'D:\8th Semester\Machine learning\ML Labs\lab2\E3-cp-0002-loss0.04.h5'
model = load_model(model_path, custom_objects={'mae': mae})

# ===============================
# 4. Set Learning Rate
# ===============================
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='mse', metrics=[mae])

# ===============================
# 5. Callbacks for Saving Checkpoints
# ===============================
checkpoints_path = r'D:\8th Semester\Machine learning\ML Labs\lab2\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
callbacks = [ModelCheckpoint(checkpoints_path, save_best_only=True, monitor='val_loss')]

# ===============================
# 6. Train Model
# ===============================
epochs = 1
batch_size = 128
verbose = 1

History = model.fit(
    train_X,
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_X, validation_y),
    callbacks=callbacks,
    verbose=verbose
)

# ===============================
# 7. Predict and Evaluate
# ===============================
y_pred_scaled = model.predict(validation_X)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_true = scaler.inverse_transform(validation_y)

# ===============================
# 8. Metrics Calculation
# ===============================
MAE_val = np.mean(abs(y_pred - y_true))
MEDAE_val = np.median(abs(y_pred - y_true))
MSE_val = np.mean(np.square(y_pred - y_true))
RMSE_val = np.sqrt(MSE_val)
MAPE_val = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
MDAPE_val = np.median(np.abs((y_true - y_pred) / y_true)) * 100

# ===============================
# 9. Print Results
# ===============================
print("Evaluation Metrics on Validation Data:")
print('MAE:', round(MAE_val, 2))
print('Median AE:', round(MEDAE_val, 2))
print('MSE:', round(MSE_val, 2))
print('RMSE:', round(RMSE_val, 2))
print('MAPE:', round(MAPE_val, 2), '%')
print('MDAPE:', round(MDAPE_val, 2), '%')


C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - loss: 0.1020 - mae: 0.2689

8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 634ms/step - loss: 0.1020 - mae: 0.2689 - val_loss: 0.0854 - val_mae: 0.2408
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 227ms/step
Evaluation Metrics on Validation Data:
MAE: 0.24
Median AE: 0.22
MSE: 0.08
RMSE: 0.29
MAPE: 378.08 %
MDAPE: 41.94 %


In [48]:

#model = load_model(r'D:\8th Semester\Machine learning\ML Labs\lab2\E3-cp-0002-loss0.04.h5')
model = load_model(r'D:\8th Semester\Machine learning\ML Labs\lab2\E3-cp-0002-loss0.04.h5', custom_objects={'mae': mean_absolute_error})
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 202ms/step
Mean Absolute Error (MAE): 0.27
Median Absolute Error (MedAE): 0.25
Mean Squared Error (MSE): 0.11
Root Mean Squared Error (RMSE): 0.33
Mean Absolute Percentage Error (MAPE): 397.29 %
Median Absolute Percentage Error (MDAPE): 43.37 %


y_test_unscaled.shape=  (100, 1)
y_pred.shape=  (100, 24)
